### Задание 1

Для датафрейма log создайте столбец source_type с разметкой источников из traffic_source:
* yandex или google - 'organic'
* paid и email из России - 'ad'
* paid и email не из России - 'other'
* остальные - из traffic_source без изменений

In [2]:
import pandas as pd

In [7]:
log = pd.read_csv('ml-latest-small/visit_log.csv', sep=';')
log.head(2)

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct


In [15]:
def source_markup(row):
    if row.traffic_source == 'yandex' or row.traffic_source == 'google':
        return 'organic'
    elif ((row.traffic_source == 'paid' or row.traffic_source == 'email') and row.region == 'Russia'):
        return 'ad'
    elif ((row.traffic_source == 'paid' or row.traffic_source == 'email') and row.region != 'Russia'):
        return 'other'
    else:
        return row.traffic_source

In [26]:
log['source_type'] = log.apply(source_markup, axis = 1)

In [31]:
# проверка работы функции
log[ log.source_type != 'other' ][log.source_type != 'ad' ][log.source_type != 'organic' ].head(5)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,timestamp,visit_id,url,region,user_id,traffic_source,source_type
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,direct
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,direct
10,1549980777,01d448afd9,https://host.ru/a5dda93e70318570c0,India,27e76a4a80,direct,direct
13,1549980792,dd46c3eebb,https://host.ru/3004a8273caeef2867,China,7e301c8c41,direct,direct


### Задание 2

Задача: отфильтровать файл URLs.txt по адресам страниц с текстами новостей.

Выполните следующие действия:
1. Прочитайте содержимое файла с датафрейм
2. Отфильтруйте страницы, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном
> Шаблон страницы новостей имеет внутри url: /, затем 8 цифр, затем дефис.

In [40]:
import re

In [135]:
log = pd.read_table('ml-latest-small/URLs.txt', sep='\t+', engine='python')
#log.head()

In [134]:
w = log [log.url.str.contains(r'/\d{8}-')].reset_index(drop = True)
#w.head()

In [133]:
q = w.url.str.extractall(r'(\d{8})').reset_index()

In [132]:
join = w.merge(q, how = 'left', left_index=True, right_index=True).filter(items = ['url', 0])
join.rename(columns={0: 'template'}, inplace=True)
join.head()
#data

,url,template
0,/politics/36188461-s-marta-zhizn-rossiyan-susc...,36188461
1,/world/36007585-tramp-pridumal-kak-reshit-ukra...,36007585
2,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...,36157853
3,/video/36001498-poyavilis-pervye-podrobnosti-g...,36001498
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...,36007585


### Домашнее задание 3

* в датафрейме data создайте столбец lemmas
* вычислите леммы запросов из столбца keyword
* леммы в столбец lemmas
> Леммы должны иметь строковый тип.

In [137]:
data = pd.DataFrame({
    'keyword': ['курс гривны к рублю', 'доллары в рубли', '100 долларов в рублях', 'курс рубля'],
    'shows': [125076, 114173, 97534, 53546],
})

In [138]:
data

,keyword,shows
0,курс гривны к рублю,125076
1,доллары в рубли,114173
2,100 долларов в рублях,97534
3,курс рубля,53546


In [140]:
from pymystem3 import Mystem

In [151]:
def calculation_lemmas(keyword):
    m = Mystem()
    lemmas = m.lemmatize(keyword)
    return ' '.join(lemmas)

In [152]:
data['lemmas'] = data.keyword.apply(calculation_lemmas)

In [154]:
data

,keyword,shows,lemmas
0,курс гривны к рублю,125076,курс гривна к рубль \n
1,доллары в рубли,114173,доллар в рубль \n
2,100 долларов в рублях,97534,100 доллар в рубль \n
3,курс рубля,53546,курс рубль \n
